In [1]:
from bs4 import BeautifulSoup,SoupStrainer
import requests
import pandas as pd
import re
import math
from pprint import pprint
from datetime import datetime

import selenium
from selenium.webdriver.support.wait import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
 
import time

In [6]:
def return_num_results(buyacar_search, results_per_page):
    
    # Set up the Selenium driver    
    driver = webdriver.Chrome(options=chrome_options) #Establish a Selenium driver

    #Go to the search results. We have to load it using selenium. Using requests does not seem to work, presumably because the content is dynamic
    driver.get(buyacar_search)

    # Establish how many results there are to search
    content = driver.find_element_by_class_name('results_total') #Find the results tag
    content.get_attribute('innerHTML') # Retrieve HTML from tag
    results_soup = BeautifulSoup(content.get_attribute('innerHTML'), 'html.parser').find("div")
    num_results=int(re.search('(\d{0,6}) Results',results_soup.get_text().replace(',','')).group(1))

    # Calculate how many results pages needed to loop through given num results specified per page
    num_pages=math.ceil(num_results/results_per_page) 
    driver.quit()

    return num_pages

def wait_for_class_to_be_available(total_wait=100):
    try:
        # Give only one class name, if you want to check multiple classes then 'and' will be use in XPATH
        # e.g //*[contains(@class, "class_name") and contains(@class, "second_class_name")]
        elem = driver.find_elements_by_class_name('search_result')
    except:
        total_wait -= 1
        time.sleep(1)
        if total_wait > 1: wait_for_class_to_be_available(total_wait)
        else:
            elem=None
    return elem

def get_search_results(num_pages, buyacar_search, results_per_page):
    #for i in range(0,num_pages):
    my_list_of_cars=[]

    for p_num in range(0,num_pages):
    #for p_num in range(0,1):

        search_url=buyacar_search+'&offset={}&limit={}'.format(p_num,results_per_page) #establish the URL of the specific page
        driver = webdriver.Chrome(options=chrome_options) #Establish a Selenium driver

        #Go to the search results
        driver.get(search_url)

        #search_results = driver.find_elements_by_class_name('search_result') #List all search results on page (length of list should )
        #search_results=wait_for_class_to_be_available()

        # Load the search results in a selenium browser. If it doesn't manage to find the path, wait until it loads
        time_count=0
        search_results=[]
        while len(search_results)==0 and time_count<40:
            search_results = driver.find_elements_by_class_name('search_result')
            if len(search_results)==0:
                time.sleep(1)
                time_count+=1
                print("Waiting...")
        if search_results==None:
            print("No search results returned for page {} @ {}".format(p_num,datetime.now()))
        elif len(search_results)==0:
            print("No search results returned for page {} @ {}".format(p_num,datetime.now()))
        else:
            print("Retrieved {} search results from page {} @ {}".format(len(search_results),p_num,datetime.now()))



        # Loop through each individual search result
        #for search_result in search_results[:1]:
        for n,search_result in enumerate(search_results):

            #if n%20==0:
            #    print("Onto result number:{} @ {}".format(n,datetime.now()))

            my_dict={}
            results_html=BeautifulSoup(search_result.get_attribute('innerHTML'), 'html.parser')

            my_dict['page']=p_num
            my_dict['search result num']=n

            #Get URL
            try:
                my_dict['link_to_car']='https://www.buyacar.co.uk/{}'.format(results_html.find("a").get("href"))
            except:
                pass

            #Get the car's detailed name
            try:
                my_dict['car name detail']=results_html.find("h2",{"class":"search_result_car_name"}).find("a").get_text()
            except:
                pass

            if 'estate' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Estate'
            elif 'suv' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='SUV'
            elif 'hatchback' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Hatchback'
            elif 'mpv' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='MPV'
            elif 'saloon' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Saloon'
            elif 'coupe' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Coupe'
            elif 'convertible' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Convertible'
            elif 'roadster' in my_dict['car name detail'].lower():
                my_dict['Body Type (Derived)']='Convertible'
            else:
                my_dict['Body Type (Derived)']='NOT CLASSIFIED'

            #Get the car's simple name
            try:
                my_dict['car name']=' '.join([x["content"] for x in results_html.find_all("meta")])
            except:
                pass


            try:
                my_dict['price']=results_html.find("div",{"class":"search_result_price"}).find("span",{"class":"one_off_price"}).get_text()
            except:
                pass
            try:
                my_dict['transmission']=results_html.find("ul",{"class":"search_result_info_icons"}).find("li",{"class":"gear_info"}).get_text()
            except:
                pass

            try:
                my_dict['colour']=results_html.find("ul",{"class":"search_result_info_icons"}).find("li",{"class":"color_info"}).get_text()
            except:
                pass
            try:
                my_dict['year']=results_html.find("ul",{"class":"search_result_info_icons"}).find("li",{"class":"date_info"}).get_text()
            except:
                pass

            try:
                my_dict['petrol type']=results_html.find("ul",{"class":"search_result_info_icons"}).find("li",{"class":"fuel_info"}).get_text()
            except:
                pass

            try:
                my_dict['mileage']=results_html.find("ul",{"class":"search_result_info_icons"}).find("li",{"class":"miles_info"}).get_text()
            except:
                pass

            ######### Go to the specific listing page for that car
            specific_car_soup=None
            try:
                req = requests.get(my_dict['link_to_car'])
                specific_car_soup = BeautifulSoup(req.content, 'html.parser')
            except:
                pass

            if specific_car_soup:
                #Check if it has isofix properties
                try:
                    int_attrs=[list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.interior-features li")]
                    my_dict['isofix']=max([1 if 'isofix' in x else 0 for x in int_attrs])
                except:
                    pass

                # Get performance info
                for i in specific_car_soup.select("ul.performance li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass

                # Get Weight and capabilities info
                for i in specific_car_soup.select("ul.weight-and-capacities li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass

                # Get General info
                for i in specific_car_soup.select("ul.general li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass

                #Get Vehicle dimensions inc. capacity of boot
                for i in specific_car_soup.select("ul.vehicle-dimensions li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass

                #Get Engine and Drive Train info (who knows, maybe it'll be useful)
                for i in specific_car_soup.select("ul.engine-and-drive-train li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass

                #Get Tyres info (who knows, maybe it'll be useful)
                for i in specific_car_soup.select("ul.tyres li"):
                    try:
                        attr=list(i.findAll("span"))
                        my_dict[attr[0].get_text()]=attr[1].get_text()
                    except:
                        pass


                #List Standard Equipment: Security
                try:
                    my_dict['Security Features']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.security li")])
                except:
                    pass

                #List Standard Equipment: Driver Convenience
                try:
                    my_dict['Entertainment Features']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.entertainment li")])
                except:
                    pass


                #List Standard Equipment: Safety
                try:
                    my_dict['Safety features']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.safety li")])
                except:
                    pass

                #List Standard Equipment: Exterior
                try:
                    my_dict['Exterior features']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.exterior-features li")])
                except:
                    pass

                #List Standard Equipment: Interior
                try:
                    my_dict['Interior features']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.interior-features li")])
                except:
                    pass

                #List Standard Equipment: Driver Convenience
                try:
                    my_dict['Driver Convenience']=safety_attrs=', '.join([list(i.findAll("span"))[0].get_text().lower() for i in specific_car_soup.select("ul.driver-convenience li")])
                except:
                    pass

            my_list_of_cars.append(my_dict)

        cars_should_have=(p_num+1)*results_per_page
        cars_do_have=len(my_list_of_cars)
        cars_delta=cars_should_have-cars_do_have

        print("There are now {} cars in the list. We should have {}, so are missing {}.".format(cars_do_have,cars_should_have,cars_delta))
        driver.quit()
        return my_list_of_cars


In [9]:
# List search URLs to review
search_results=[\
'https://www.buyacar.co.uk/cars?f[0]=edition:Ceed%20Sportswagon&f[1]=fuel:Petrol&f[2]=gearbox:manual&f[3]=make:KIA&f[4]=mileage:%5B*%20TO%2030000%5D&f[5]=model:Ceed&f[6]=price:%5B7000%20TO%2014000%5D&f[7]=vehicle_type:car'\
,'https://www.buyacar.co.uk/cars?f[0]=edition:CR-V%20Estate&f[1]=fuel:Petrol&f[2]=gearbox:manual&f[3]=make:Honda&f[4]=mileage:%5B10000%20TO%2030000%5D&f[5]=model:CR-V&f[6]=price:%5B9000%20TO%2014000%5D&f[7]=vehicle_type:car']

# Input results per page
try:
    results_per_page=int(input('And how many results should be loaded per page (max 100)?'))
except:
    results_per_page=100 # If it fails, overwrite with 100 as default

# Set up options for Selenium
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=C:/Users/Steve/AppData/Local/Google/Chrome/User Data/Default") 

    
combined_list=[]
for i in search_results:
    n=return_num_results(i,results_per_page)
    search_list=get_search_results(n,i,results_per_page)
    combined_list=combined_list+search_list

And how many results should be loaded per page (max 100)?100


C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: use options instead of chrome_options


Retrieved 9 search results from page 0 @ 2020-06-06 12:29:37.805995
There are now 9 cars in the list. We should have 100, so are missing 91.
Retrieved 8 search results from page 0 @ 2020-06-06 12:30:07.894220
There are now 8 cars in the list. We should have 100, so are missing 92.


In [11]:
pd.options.display.max_rows=9999
pd.options.display.max_columns=40

In [12]:
cars_df=pd.DataFrame(combined_list)

In [14]:
filtered_car_df=cars_df.copy()
filtered_car_df.shape


(17, 88)

In [15]:
# Derive brand name as the first bit of data before a space
filtered_car_df['Brand']=filtered_car_df['car name'].str.split(' ').apply(lambda x: x[0])
filtered_car_df['Make exc. Year']=filtered_car_df['car name'].str.split(' ').apply(lambda x: ' '.join(x[:-1]))
filtered_car_df.shape

(17, 90)

In [16]:
filtered_car_df['Brand'].unique()

array(['KIA', 'Honda'], dtype=object)

In [19]:
filtered_car_df=filtered_car_df[['Brand','Make exc. Year','car name','car name detail','Body Type (Derived)','colour','isofix',\
                 'link_to_car','mileage','page','petrol type','price','transmission','Luggage Capacity (Seats Up)','year',\
                                 'Badge Engine CC','Fuel Tank Capacity (Litres)','Gears','Height','Length','Width','NCAP Child Occupant Protection %',\
                                'Insurance Group 1 - 50 Effective January 07',\
                                'Driver Convenience','Entertainment Features','Exterior features','Interior features','Safety features','Security Features']]
filtered_car_df.shape

(17, 29)

In [20]:
filtered_car_df['Volume']=pd.to_numeric(filtered_car_df['Height'])*\
pd.to_numeric(filtered_car_df['Width'])*\
pd.to_numeric(filtered_car_df['Length'])
filtered_car_df.shape

C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(17, 30)

In [21]:
filtered_car_df['Price £s']=pd.to_numeric(filtered_car_df['price'].str.replace(r"[\D]",'')) # Remove all non numeric characters
filtered_car_df['Mileage']=pd.to_numeric(filtered_car_df['mileage'].str.replace(r"[\D]",'')) # Remove all non numeric characters
filtered_car_df['Year']=pd.to_numeric(filtered_car_df['year'].str[:4]) # Remove all non numeric characters
filtered_car_df['Year (Plate)']=pd.to_numeric(filtered_car_df['year'].str[-3:-1]) # Remove all non numeric characters
filtered_car_df['Boot Space']=pd.to_numeric(filtered_car_df['Luggage Capacity (Seats Up)'])
filtered_car_df.shape

C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

(17, 35)

In [22]:
filtered_car_df

,Brand,Make exc. Year,car name,car name detail,Body Type (Derived),colour,isofix,link_to_car,mileage,page,petrol type,price,transmission,Luggage Capacity (Seats Up),year,Badge Engine CC,Fuel Tank Capacity (Litres),Gears,Height,Length,Width,NCAP Child Occupant Protection %,Insurance Group 1 - 50 Effective January 07,Driver Convenience,Entertainment Features,Exterior features,Interior features,Safety features,Security Features,Volume,Price £s,Mileage,Year,Year (Plate),Boot Space
0,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 1.0T GDi ISG 2 5dr sportswagon,NOT CLASSIFIED,White,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"16,438 miles",0,Petrol,"£13,350",manual,625,2019 (68),1.0,50,6 SPEED,1465,4600,1800,NaN,12D,"3.5"" supervision cluster display, 7"" colour to...","6 speaker audio system, dab radio with mp3, il...",all-round electric windows with front auto up ...,"12v power socket in front console (x2), 3 x pa...",abs with electronic brakeforce distribution (e...,"alarm system, engine immobiliser & deadlocks, ...",12130200000,13350,16438,2019,68,625
1,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 1.0T GDi ISG 2 5dr sportswagon,NOT CLASSIFIED,Black,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"13,465 miles",0,Petrol,"£13,750",manual,625,2019 (19),1.0,50,6 SPEED,1465,4600,1800,NaN,8E,"3.5"" supervision cluster display, 7"" colour to...","6 speaker audio system, dab radio with mp3, il...",all-round electric windows with front auto up ...,"12v power socket in front console (x2), 3 x pa...",abs with electronic brakeforce distribution (e...,"alarm system, engine immobiliser & deadlocks, ...",12130200000,13750,13465,2019,19,625
2,KIA,KIA Ceed,KIA Ceed 2016,KIA CEED 1.0 3 ISG estate sportswagon,Estate,Silver,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"13,975 miles",0,Petrol,"£9,995",manual,510,2016 (66),1.0,53,6 SPEED,1485,4505,1780,88,11E,"7"" touchscreen full map navigation system with...","6 speakers, bluetooth audio streaming, dab rad...","auto dimming rear view mirror, automatic defog...","12v power socket in front console (x2), 12v so...","3 point seatbelts on all seats, abs/ebd, autom...","alarm system, deadlocks, immobiliser, locking ...",11908066500,9995,13975,2016,66,510
3,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 1.0T GDI ISG 2 SPORTSWAGON ESTATE estate,Estate,White,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"11,536 miles",0,Petrol,"£12,495",manual,625,2019 (68),1.0,50,6 SPEED,1465,4600,1800,NaN,12D,"3.5"" supervision cluster display, 7"" colour to...","6 speaker audio system, dab radio with mp3, il...",all-round electric windows with front auto up ...,"12v power socket in front console (x2), 3 x pa...",abs with electronic brakeforce distribution (e...,"alarm system, engine immobiliser & deadlocks, ...",12130200000,12495,11536,2019,68,625
4,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 1.0T GDi ISG 2 5dr sportswagon,NOT CLASSIFIED,Silver,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"10,483 miles",0,Petrol,"£13,850",manual,625,2019 (19),1.0,50,6 SPEED,1465,4600,1800,NaN,8E,"3.5"" supervision cluster display, 7"" colour to...","6 speaker audio system, dab radio with mp3, il...",all-round electric windows with front auto up ...,"12v power socket in front console (x2), 3 x pa...",abs with electronic brakeforce distribution (e...,"alarm system, engine immobiliser & deadlocks, ...",12130200000,13850,10483,2019,19,625
5,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 2 ISG estate sportswagon,Estate,Grey,1,https://www.buyacar.co.uk//kia/ceed/ceed-sport...,"11,925 miles",0,Petrol,"£13,995",manual,625,2019 (19),1.0,50,6 SPEED,1465,4600,1800,NaN,8E,"3.5"" supervision cluster display, 7"" colour to...","6 speaker audio system, dab radio with mp3, il...",all-round electric windows with front auto up ...,"12v power socket in front console (x2), 3 x pa...",abs with electronic brakeforce distribution (e...,"alarm system, engine immobiliser & deadlocks, ...",12130200000,13995,11925,2019,19,625
6,KIA,KIA Ceed,KIA Ceed 2019,KIA CEED 2 ISG sportswagon,

In [23]:
filtered_car_df.to_excel(r"C:\Users\steve\Dropbox\Cars (Final Two).xlsx")